In [8]:
from collections import defaultdict
from collections import deque


In [4]:
import networkx as nx
G = nx.read_adjlist('graph0.adjlist', nodetype=int)

In [22]:
# G.add_edge(43,156)
# betweenness_centrality = max(nx.betweenness_centrality(G,normalized=False).values())
# print(betweenness_centrality)

In [84]:
from collections import defaultdict

def stress_centrality0(G):
    stress = defaultdict(int)
    nodes = list(G.nodes())
    for s in nodes:
        for t in nodes:
            if s != t:
                paths = list(nx.all_shortest_paths(G, s, t))
                for path in paths:
                    for v in path[1:-1]:
                        stress[v] += 1
    return stress

print(stress_centrality0(G))

defaultdict(<class 'int'>, {1: 4, 3: 4, 2: 10, 0: 2})


In [109]:
from collections import deque

def stress_centrality(G):
    stress = {n: 0 for n in G.nodes()}
    nodes = G.nodes()
    
    for s in nodes:
        S = []
        P = {w: [] for w in nodes}
        sigma = {w: 0 for w in nodes}
        sigma[s] = 1
        d = {w: -1 for w in nodes}
        d[s] = 0
        Q = deque([s])
        
        # Breadth-first search
        while Q:
            v = Q.popleft()
            S.append(v)
            for w in G.neighbors(v):
                # Path discovery
                if d[w] < 0:
                    Q.append(w)
                    d[w] = d[v] + 1
                if d[w] == d[v] + 1:
                    sigma[w]+=sigma[v]
                    P[w].append(v)
        
        delta = {w: 0 for w in nodes}
        while S:
            w = S.pop()
            for v in P[w]:
                delta[v]+=(delta[w]+1)
            if w != s:
                stress[w]+=delta[w]*sigma[w]/2
        print(w,delta,sigma)
    
    return stress

In [110]:
G = nx.read_adjlist('graph0.adjlist', nodetype=int)
stress = stress_centrality(G)
print(stress)
for i in G.nodes():
    for j in G.nodes():
        if i < j and not G.has_edge(i, j):
            G.add_edge(i, j)
            print(i,j,stress_centrality(G))
            G.remove_edge(i, j)

0 {0: 4, 1: 3, 2: 2, 3: 0, 4: 0} {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
1 {0: 0, 1: 4, 2: 2, 3: 0, 4: 0} {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
2 {0: 0, 1: 1, 2: 4, 3: 0, 4: 0} {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
3 {0: 0, 1: 1, 2: 3, 3: 4, 4: 0} {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
4 {0: 0, 1: 1, 2: 3, 3: 0, 4: 4} {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: 0.0, 1: 3.0, 2: 5.0, 3: 0.0, 4: 0.0}
0 {0: 4, 1: 0, 2: 2, 3: 0, 4: 0} {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
1 {0: 0, 1: 4, 2: 2, 3: 0, 4: 0} {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
2 {0: 0, 1: 0, 2: 4, 3: 0, 4: 0} {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
3 {0: 0, 1: 0, 2: 3, 3: 4, 4: 0} {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
4 {0: 0, 1: 0, 2: 3, 3: 0, 4: 4} {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
0 2 {0: 0.0, 1: 0.0, 2: 5.0, 3: 0.0, 4: 0.0}
0 {0: 6, 1: 2, 2: 1, 3: 2, 4: 0} {0: 1, 1: 1, 2: 2, 3: 1, 4: 2}
1 {0: 1, 1: 5, 2: 2, 3: 0, 4: 0} {0: 1, 1: 1, 2: 1, 3: 2, 4: 1}
2 {0: 0, 1: 1, 2: 5, 3: 1, 4: 0} {0: 2, 1: 1, 2: 1, 3: 1, 4: 1}
3 {0: 1, 1: 0, 2: 2, 3: 5, 4: 0} {0: 1, 1: 2, 2: 1, 3: 1, 4: 1}
4 {0: 0, 1: 1, 2: 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
nx.draw(G, with_labels=True, node_color='lightblue', edge_color='gray', node_size=500, font_size=10)
plt.title("Graph Visualization")
plt.show()

In [1]:
def stress_centrality_brute_force(G):
    stress_centrality = defaultdict(int)
    nodes = sorted(G.nodes())      
    for i, source in enumerate(nodes):
        for target in nodes[i+1:]:  
            try:
                paths = list(nx.all_shortest_paths(G, source=source, target=target))
                for path in paths:
                    for node in path[1:-1]:  # Exclude source and target
                        stress_centrality[node] += 1
            except nx.NetworkXNoPath:
                continue
    return stress_centrality

In [12]:
def cmp(filename):
    G = nx.read_adjlist(filename, nodetype=int)
    ans = max(stress_centrality_brute_force(G).values())
    print(ans)

In [13]:
for i in range(0,6):
    cmp('graph'+str(i)+'.adjlist')

5
12864
7767
14194
8808
12448
